# Runs for Micro-wrapper development

In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY311,
)

In [2]:
import os
import subprocess
import sys

sys.path.insert(1, os.path.abspath(".."))

import GooseSLURM as gs
import numpy as np

from pint import Quantity

import lysis
from lysis.util import Q_

In [3]:
#############################################
#### TODO: Change all this to dataframes that read from CSV
#############################################

scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", Quantity),  # in microns. Code requires cm
        (
            "fiber_diameter",
            Quantity,
        ),  # in nanometers. Needs to be added to pore_size to get grid_node_distance
        # ("binding_sites", Quantity),
        ("cols", int),
        ("rows", int),
        ("empty_rows", int),
        ("forced_unbind", float),
        ("average_bound_time", Quantity),
        ("nodes", int),
        ("diss_const_tPA_wPLG", Quantity),
        ("diss_const_tPA_woPLG", Quantity),
    ]
)
mechanism_type = np.dtype(
    [("descriptor", np.str_, 40), ("micro_executable", np.str_, 40), ("macro_executable", np.str_, 40)]
)
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("source_exp", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("micro_seed", np.int64),
        ("macro_seed", np.int64),
    ]
)
lysis_root = os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis")

In [4]:
scenarios = np.array(
    [
        (
            "0.1-PKd",
            43_074,
            Q_("1.0135 um"),
            Q_("72.7 nm"),
            # Q_("427 uM"),
            93,
            121,
            28,
            0.5143,
            Q_("277.8 sec"),
            7,
            Q_("0.002 uM"),
            Q_("0.036 uM"),
        ),
        (
            "1-PKd",
            43_074,
            Q_("1.0135 um"),
            Q_("72.7 nm"),
            # Q_("427 uM"),
            93,
            121,
            28,
            8.52e-2,
            Q_("27.8 sec"),
            7,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "10-PKd",
            43_074,
            Q_("1.0135 um"),
            Q_("72.7 nm"),
            # Q_("427 uM"),
            93,
            121,
            28,
            5.4e-3,
            Q_("2.78 sec"),
            7,
            Q_("0.2 uM"),
            Q_("3.6 uM"),
        ),
        (
            "TN-L__9_350",
            9_350,
            Q_("1.0135 um"),
            Q_("72.7 nm"),
            # Q_("427 uM"),
            93,
            93,
            0,
            0.0852,
            Q_("27.8 sec"),
            7,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TN-D__9_350",
            9_350,
            Q_("0.22 um"),
            Q_("72.7 nm"),
            # Q_("427 uM"),
            342,
            116,
            0,
            0.0852,
            Q_("27.8 sec"),
            7,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TK-L__9_350",
            9_350,
            Q_("1.0135 um"),
            Q_("145.4 nm"),
            # Q_("213 uM"),
            87,
            29,
            0,
            0.0729129,
            Q_("27.8 sec"),
            13,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TK-D__9_350",
            9_350,
            Q_("0.22 um"),
            Q_("145.4 nm"),
            # Q_("213 uM"),
            274,
            42,
            0,
            0.0729129,
            Q_("27.8 sec"),
            13,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TB-xi__21_105",
            21_105,
            Q_("5.34 um"),
            Q_("123 nm"),
            # Q_("248.531 uM"),
            19,
            184,
            83,
            0.077725,
            Q_("27.8 sec"),
            11,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TF-x__9_951",
            9_951,
            Q_("2.5 um"),
            Q_("105.1 nm"),
            # Q_("306.357 uM"),
            39,
            385,
            326,
            0.078644,
            Q_("27.8 sec"),
            10,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TF-vii__9_951",
            9_951,
            Q_("2.5 um"),
            Q_("105.1 nm"),
            # Q_("204.238 uM"),
            39,
            385,
            265,
            0.086698,
            Q_("27.8 sec"),
            7,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
        (
            "TF-v__9_951",
            9_951,
            Q_("2.5 um"),
            Q_("105.1 nm"),
            # Q_("136.159 uM"),
            39,
            385,
            151,
            0.084703,
            Q_("27.8 sec"),
            5,
            Q_("0.02 uM"),
            Q_("0.36 uM"),
        ),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - External", "micro_rates", "macro_diffuse_into_and_along__external"),
        ("Into and along - Internal", "micro_rates", "macro_diffuse_into_and_along__internal"),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_{scenario_code}"
out_file_code = "_{scenario_code}"

In [6]:
group_code = "2024-09-02-14"
runs = np.array(
    [
        (
            group_code + "00",
            "2023-02-02-2203",
            "1-PKd",
            "Into and along - External",
            981_681_759,
            -2_137_354_075,
        ),
        (
            group_code + "01",
            "2023-02-02-2208",
            "0.1-PKd",
            "Into and along - External",
            221_452_692,
            -854_989_241,
        ),
        (
            group_code + "02",
            "2023-02-02-2213",
            "10-PKd",
            "Into and along - External",
            -1_986_027_869,
            -1_212_172_957,
        ),
        (
            group_code + "03",
            "2023-12-10-1900",
            "TN-L__9_350",
            "Into and along - Internal",
            981_681_759,
            1_034_836_197,
        ),
        (
            group_code + "04",
            "2023-12-10-1903",
            "TN-D__9_350",
            "Into and along - Internal",
            981_681_759,
            129_314_213,
        ),
        (
            group_code + "05",
            "2023-12-10-1906",
            "TK-L__9_350",
            "Into and along - Internal",
            -34_041_038,
            2_608_028_918,
        ),
        (
            group_code + "06",
            "2023-12-10-1909",
            "TK-D__9_350",
            "Into and along - Internal",
            -34_041_038,
            106_672_317,
        ),
        (
            group_code + "07",
            "2024-04-16-1912",
            "TB-xi__21_105",
            "Into and along - External",
            1_093_845_299,
            2_061_649_767,
        ),
        (
            group_code + "08",
            "2024-04-16-1913",
            "TF-x__9_951",
            "Into and along - External",
            -85_552_210,
            3_687_972_975,
        ),
        (
            group_code + "09",
            "2024-04-16-1914",
            "TF-vii__9_951",
            "Into and along - External",
            -398_618_820,
            1_422_833_478,
        ),
        (
            group_code + "10",
            "2024-04-16-1915",
            "TF-v__9_951",
            "Into and along - External",
            -150_168_862,
            77_748_149,
        ),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    print(f"{run['exp_code']}: {run['scenario']} -- {run['mechanism']}")
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    micro_param = {
        "fiber_radius": scen["fiber_diameter"] / 2,
        "nodes_in_row": int(scen["nodes"]),
        "diss_const_tPA_wPLG": scen["diss_const_tPA_wPLG"],
        "diss_const_tPA_woPLG": scen["diss_const_tPA_woPLG"],
        "seed": int(run["micro_seed"]),
        "micro_version": mech["micro_executable"],
    }
    e.initialize_micro_param(micro_param)
    macro_param = {
        "total_molecules": int(scen["total_molecules"]),
        "pore_size": scen["pore_size"],
        "cols": int(scen["cols"]),
        "rows": int(scen["rows"]),
        "empty_rows": int(scen["empty_rows"]),
        "forced_unbind": scen["forced_unbind"],
        "average_bound_time": scen["average_bound_time"],
        "seed": int(run["macro_seed"]),
        "macro_version": mech["macro_executable"],
        "total_time": Q_("0 min"),
    }
    e.initialize_macro_param(macro_param)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
Micro Wrapper Testing
Array version

Uses Fortran Microscale code.

Uses Fortran Macroscale code with
    -  Diffuse into and along
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing the termination criteria from 'fixed time' to 'all fibers degraded'
    -  changing macro-unbind wait time to remaining leaving time.
    -  eliminating unused data variables
    -  Moved degraded fiber check into molecule loop
    -  Removed "degrade" array and use "t_degrade" instead
    -  Read in "neighborc" array generated in Python
    -  Output molecule bind/unbind times for transit time calculations
    -  Output molecule bind/unbind locations.

This set of data is comparable to "{run['scenario']}" from experiment "{run['source_exp']}".
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["micro_executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)
    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["macro_executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

2024-09-02-1400: 1-PKd -- Into and along - External
CompletedProcess(args=['cp', 'src/fortran/micro_rates.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1400'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1400'], returncode=0, stdout=b'', stderr=b'')
2024-09-02-1401: 0.1-PKd -- Into and along - External
CompletedProcess(args=['cp', 'src/fortran/micro_rates.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1401'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1401'], returncode=0, stdout=b'', stderr=b'')
2024-09-02-1402: 10-PKd -- Into and along - External
CompletedProcess(args=['cp', 'src/fortran/micro_rates.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1402'], returncode

In [12]:
run = runs[0]
mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
e = lysis.util.Experiment(
    os.path.join(lysis_root, "data"),
    experiment_code=run["exp_code"],
)
e.read_file()
# f = lysis.util.FortranMicro(
#     exp=e,
#     cwd=lysis_root,
#     executable=os.path.join(lysis_root, "bin", mech["micro_executable"]),
#     out_file_code=out_file_code.format(scenario_code=run["scenario"]) + ".dat",
# )
# f.run()
print(e)

experiment_code : 2024-09-02-1400
data_filenames  : lysis_complete_time : lysis
                  tPA_leaving_time    : tPA_time
                  PLi_generated       : PLi
                  lysis_completed     : lyscomplete
                  tPA_kinetic_unbound : tPAunbind
                  tPA_forced_unbound  : tPAPLiunbind
                  tPA_still_bound     : lasttPA
                  first_PLi           : firstPLi
                  unbinding_time_dist : tsectPA
                  lysis_time_dist     : lysismat
                  total_lyses         : lenlysisvect
                  degradation_state   : deg
                  molecule_location   : m_loc
                  molecule_state      : m_bound
                  save_time           : tsave
micro_params    : fibrinogen_length          : 0.045 micron
                  fibrinogen_radius          : 0.0012 micron
                  fiber_radius               : 0.03635 micron
                  protofibril_radius         : 0.0024 micr

In [8]:
folder_vars = """
homedir="/home/bpaynter/git/UCO-OpResearch/lysis"
workdir="/tmp/bpaynter/${SLURM_JOB_ID}"
datadir="${workdir}/data/${exp_code}"
"""

transfer_script = """
# 1. Transfer to node 
# ====================
echo "Starting data transfer to node."

# create/empty the temporary directory on the compute node
if [ ! -d "${workdir}" ]; then
  mkdir -p "${workdir}"
else
  echo rm -ri "${workdir}"/*
fi

mkdir -p "${datadir}"

cp ${homedir}/data/${exp_code}/params.json ${datadir}/
cp ${homedir}/bin/${fort_executable} ${workdir}/
echo "Data transfer to node completed."
"""

return_script = """
# 2. Function to transfer back to the head node 
# ==============================================

# define clean-up function
function clean_up {
  # - copy everything from the temporary directory on the compute-node
  echo "Copying data back to home directory."
  mkdir -p ${homedir}/data/${exp_code}
  cp -prf "${datadir}"/*${out_code}* ${homedir}/data/${exp_code}/
  cp -prf "${datadir}"/*.txt ${homedir}/data/${exp_code}/
  # - erase the temporary directory from the compute-node
  echo "Erasing temporary directories."
  rm -rf "${workdir}"/*
  rm -rf "${workdir}"
  # - exit the script
  echo "Job complete. Exiting."
  exit
}

# call "clean_up" function when this script exits, it is run even if SLURM cancels the job
trap 'clean_up' EXIT
"""
execute_script = """
# 3. Execute 
# ===========

source /home/bpaynter/.bashrc
source /home/bpaynter/lysis.sh
cd ${homedir}/src/python
echo "Starting code execution."

python -u fortran_run.py \
    --out_code ${out_code}.dat \
    --cwd ${workdir} \
    ${workdir}/${fort_executable} \
    ${exp_code}
    
sleep 10
echo "Code execution complete."
"""

In [9]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join(lysis_root, "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    run_vars = f"""
exp_code="{e.experiment_code}"
out_code="{out_file_code.format(scenario_code=run['scenario'])}"
fort_executable="{mech['micro_executable']}"
echo "Experiment ${{exp_code}} Micro (${{out_code}})"
"""
    # job-options
    sbatch = {
        "job-name": f"lysis-micro-{e.experiment_code}",
        "out": os.path.join(e.os_path, "job.slurm-%A.out"),
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "exclusive=user": "",
        # "exclusive": "",
        # "partition": "long",
    }
    script = [run_vars, folder_vars, transfer_script, return_script, execute_script]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=lysis_root,
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1400/job.slurm'], returncode=0, stdout=b'Submitted batch job 2224096\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1401/job.slurm'], returncode=0, stdout=b'Submitted batch job 2224097\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1402/job.slurm'], returncode=0, stdout=b'Submitted batch job 2224098\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1403/job.slurm'], returncode=0, stdout=b'Submitted batch job 2224099\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1404/job.slurm'], returncode=0, stdout=b'Submitted batch job 2224100\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-09-02-1405/job.slurm'], returncode=0, std